In [24]:
### Data Preparation. There is need three folders: validation, train, and test, which must be divided for the task

### import some libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import os
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, precision_recall_fscore_support
import seaborn as sns
import pandas as pd
from PIL import Image
import random
import glob
from torchviz import make_dot
import matplotlib.image as mpimg
import shutil
import xml.etree.ElementTree as ET
import json


In [25]:
torch.cuda.is_available()


True

In [6]:



#%% Function for conversion XML to YOLO
# based on https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]


def create_folder(path):
    if not os.path.exists(path):
        os.makedirs(path)

create_folder('yolov7/train/images')
create_folder('yolov7/train/labels')
create_folder('yolov7/val/images')
create_folder('yolov7/val/labels')
create_folder('yolov7/test/images')
create_folder('yolov7/test/labels')

#%% get all image files
img_folder = 'images'
_, _, files = next(os.walk(img_folder))
pos = 0
for f in files:
        source_img = os.path.join(img_folder, f)
        if pos < 700:
            dest_folder = 'yolov7/train'
        elif (pos >= 700 and pos < 800):
            dest_folder = 'yolov7/val'
        else:
            dest_folder = 'yolov7/test'
        destination_img = os.path.join(dest_folder,'images', f)
        shutil.copy(source_img, destination_img)

        # check for corresponding label
        label_file_basename = os.path.splitext(f)[0]
        label_source_file = f"{label_file_basename}.xml"
        label_dest_file = f"{label_file_basename}.txt"
        
        label_source_path = os.path.join('annotations', label_source_file)
        label_dest_path = os.path.join(dest_folder, 'labels', label_dest_file)
        # if file exists, copy it to target folder
        if os.path.exists(label_source_path):
             # parse the content of the xml file
            tree = ET.parse(label_source_path)
            root = tree.getroot()
            width = int(root.find("size").find("width").text)
            height = int(root.find("size").find("height").text)
            classes = ['with_mask', 'without_mask', 'mask_weared_incorrect']
            result = []
            for obj in root.findall('object'):
                label = obj.find("name").text
                # check for new classes and append to list
                index = classes.index(label)
                pil_bbox = [int(x.text) for x in obj.find("bndbox")]
                yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
                # convert data to string
                bbox_string = " ".join([str(x) for x in yolo_bbox])
                result.append(f"{index} {bbox_string}")
                if result:
                    # generate a YOLO format text file for each xml file
                    with open(label_dest_path, "w", encoding="utf-8") as f:
                        f.write("\n".join(result))
                        
        
        pos += 1
        



In [3]:

from ultralytics import YOLO


In [4]:

# sources: 
# https://docs.ultralytics.com/cli/
# https://docs.ultralytics.com/cfg/

model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)



In [12]:


# Train the model
results = model.train(data="yolov8/train_custom/masks.yaml", epochs=10, imgsz=512, batch=4, verbose=True, device='gpu')
# device=0...GPU
#  Export the model
model.export()



Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:gpu (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=yolov8/train_custom/masks.yaml, epochs=10, time=None, patience=100, batch=4, imgsz=512, save=True, save_period=-1, cache=False, device=gpu, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_lab

100%|██████████| 755k/755k [00:00<00:00, 9.33MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/yolov8/train/labels... 356 images, 0 backgrounds, 0 corrupt: 100%|██████████| 356/356 [00:01<00:00, 317.81it/s]

train: New cache created: /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/yolov8/train/labels.cache


...

                   all         52        251      0.886      0.479      0.565       0.36



10 epochs completed in 0.022 hours.
Optimizer stripped from runs/detect/train10/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train10/weights/best.pt, 6.2MB

Validating runs/detect/train10/weights/best.pt...
Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:gpu (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:01<00:00,  6.51it/s]


                   all         52        251      0.886      0.479      0.566      0.361
             with_mask         46        181      0.888      0.834      0.894      0.613
          without_mask         16         53      0.769      0.604      0.696      0.406
 mask_weared_incorrect         11         17          1          0      0.109     0.0642
Speed: 0.3ms preprocess, 2.2ms inference, 0.0ms loss, 7.6ms postprocess per image
Results saved to runs/detect/train10
Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:gpu (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/train10/weights/best.pt' with input shape (1, 3, 512, 512) BCHW and output shape(s) (1, 7, 5376) (5.9 MB)

TorchScript: starting export with torch 2.3.1+cu121...
TorchScript: export success ✅ 0.8s, saved as 'runs/detect/train10/weights/best.torchscript' (11.9 MB)

Export complete (1.0s)
Resul

'runs/detect/train10/weights/best.torchscript'

In [28]:
!yolo detect predict model= runs/detect/train10/weights/best.pt source=Video.mp4 conf=0.3 device="0"

Ultralytics YOLOv8.2.55 🚀 Python-3.10.12 torch-2.3.1+cu121 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs

video 1/1 (frame 1/927) /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/Video.mp4: 512x288 1 without_mask, 123.3ms
video 1/1 (frame 2/927) /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/Video.mp4: 512x288 1 without_mask, 10.1ms
video 1/1 (frame 3/927) /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/Video.mp4: 512x288 1 without_mask, 8.7ms
video 1/1 (frame 4/927) /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/Video.mp4: 512x288 1 without_mask, 6.8ms
video 1/1 (frame 5/927) /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/Video.mp4: 512x288 1 without_mask, 13.9ms
video 1/1 (frame 6/927) /home/hernan/Pytorch_scripts/CNN/ObjectDetection/Mask_detection/Video.mp4: 512x288 1 without_mask, 13.8ms
video 1/1 (frame 7/927) /ho